In [1]:
from pprint import pprint
import base64, requests, sys, os, json
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

In [2]:
# DATA LAKE
CLIENT_ID = '66a08b11-c3c4-4a45-9912-2b42ffc9bc25'
CLIENT_SECRET = 'msJj0hjryI4uW3sZdMfA-6P6SeP4jSh9VylYN2297rM'

#### michal winiarski
CLIENT_ID = '1d9f9c0e-d7ca-413d-bcac-103b3cd63f74'

CLIENT_SECRET = 'y7LWZsfBNjB0VlTvEpkC1t7DGSV3HIh2Ga6p_zLC4X0'

# get a report

In [3]:
data_to_post = {
   "name": "Export Test POST",
   "quartzCronExpression": "0 14 * * * ?",
   "nextFireTime": "2022-07-07T16:14:00.000Z",
   "dateCreated": "2022-07-07T08:04:01.363Z",
   "dateModified": "2022-07-07T14:27:59.446Z",
   "description": "Export Test 2",
   "timeZone": "Europe/Warsaw",
   "timePeriod": "USE_INTERVAL",
   "interval": "2022-07-05T22:00:00.000Z/2022-07-06T22:00:00.000Z",
   "reportFormat": "XLS",
   "locale": "en_US",
   "enabled": True,
   "reportId": "03bc1eef-082e-46c1-b9a8-fe45fbc3b205",
   "parameters": {"queueIds": [
          "780807e6-83b9-44be-aff0-a41c37fab004",
          "383ee5e5-5d8f-406c-ad53-835b69fe82c5",
          "d4fa7080-63d3-47ea-bcde-a4f89ae2870c",
          "427b6226-cafd-4425-ba20-4e7dd8561088",
          "e0efef7f-b61a-4caf-b71b-002595c6899e",
          "b44af752-2ce4-4810-8b83-6d131f493e23",
          "247586a2-986f-4078-b28e-e0a5694ec726"
        ],
        "mediaTypes": []},
   "lastRun": {
      "name": "",
      "reportId": "",
      "runTime": "",
      "runStatus": "",
      "errorMessage": "",
      "runDurationMsec": 0,
      "reportUrl": "",
      "reportFormat": "",
      "scheduleUri": ""
   }
}
print(type(data_to_post))

print(type(json.dumps(data_to_post)))

<class 'dict'>
<class 'str'>


### create a token

In [4]:
ENVIRONMENT = 'mypurecloud.de'
url = f"https://login.{ENVIRONMENT}/oauth/token"
print(f"URL: {url}")

# Base64 encode the client ID and client secret
authorization = base64.b64encode(bytes(CLIENT_ID + ":" + CLIENT_SECRET, "ISO-8859-1")).decode("ascii")
# print(f"AUTHORIZATION: {authorization}")

URL: https://login.mypurecloud.de/oauth/token


In [5]:
# Prepare for POST /oauth/token request
request_headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded"
}
request_body = {
    "grant_type": "client_credentials"
}

# Get token
response = requests.post(url, data=request_body, headers=request_headers)

# Check response
if response.status_code == 200:
    print("Got token")
else:
    print(f"Failure: { str(response.status_code) } - { response.reason }")
    sys.exit(response.status_code)

# Get JSON response body
response_json = response.json()

Got token


### send credentials and the report schedule

In [6]:
# Once we post to get the token, post all data we want
requestHeaders = {
    "Authorization": f"{ response_json['token_type'] } { response_json['access_token']}",
    'Content-Type': 'application/json'
}

url = f"https://api.{ENVIRONMENT}/api/v2/analytics/reporting/schedules"

response = requests.post(url, headers=requestHeaders, data=json.dumps(data_to_post))

### get the response

In [7]:
# Prepare for GET /api/v2/authorization/roles request
requestHeaders = {
    "Authorization": f"{ response_json['token_type'] } { response_json['access_token']}",
    "Content-Type": "application/json"
}

url = f"https://api.{ENVIRONMENT}/api/v2/analytics/reporting/schedules"
print(url, end="\n\n")

response = requests.get(url, headers=requestHeaders)

# Check response
if response.status_code == 200:
    print("Got reporting/schedules")
else:
    print(f"Failure: { str(response.status_code) } - { response.reason }")


r = response.json()

https://api.mypurecloud.de/api/v2/analytics/reporting/schedules

Got reporting/schedules


### print the result in a friendly way

In [8]:
r

{'entities': [{'id': 'c8c88e75-8f46-4b23-bae5-7ff8308ac00d',
   'quartzCronExpression': '0 14 * * * ?',
   'nextFireTime': '2022-07-14T10:14:00.000Z',
   'dateCreated': '2022-07-14T08:16:09.811Z',
   'dateModified': '2022-07-14T08:16:09.811Z',
   'description': 'Export Test 2',
   'timeZone': 'Europe/Warsaw',
   'timePeriod': 'USE_INTERVAL',
   'interval': '2022-07-05T22:00:00.000Z/2022-07-06T22:00:00.000Z',
   'reportFormat': 'XLS',
   'locale': 'en_US',
   'enabled': True,
   'reportId': '03bc1eef-082e-46c1-b9a8-fe45fbc3b205',
   'parameters': {'queueIds': ['780807e6-83b9-44be-aff0-a41c37fab004',
     '383ee5e5-5d8f-406c-ad53-835b69fe82c5',
     'd4fa7080-63d3-47ea-bcde-a4f89ae2870c',
     '427b6226-cafd-4425-ba20-4e7dd8561088',
     'e0efef7f-b61a-4caf-b71b-002595c6899e'],
    'mediaTypes': []},
   'selfUri': '/api/v2/analytics/reporting/schedules/c8c88e75-8f46-4b23-bae5-7ff8308ac00d'},
  {'id': '0cdd3c66-27ad-4d53-b64d-169611d6cc86',
   'quartzCronExpression': '0 14 * * * ?',
   'n

In [9]:
tree_str = json.dumps(r, indent=4)
tree_str = tree_str.replace("\n    ", "\n")
tree_str = tree_str.replace('"', "")
tree_str = tree_str.replace(',', "")
tree_str = tree_str.replace("{", "")
tree_str = tree_str.replace("}", "")
tree_str = tree_str.replace("    ", " | ")
tree_str = tree_str.replace("  ", " ")

print(tree_str)


entities: [
 | 
 | | id: c8c88e75-8f46-4b23-bae5-7ff8308ac00d
 | | quartzCronExpression: 0 14 * * * ?
 | | nextFireTime: 2022-07-14T10:14:00.000Z
 | | dateCreated: 2022-07-14T08:16:09.811Z
 | | dateModified: 2022-07-14T08:16:09.811Z
 | | description: Export Test 2
 | | timeZone: Europe/Warsaw
 | | timePeriod: USE_INTERVAL
 | | interval: 2022-07-05T22:00:00.000Z/2022-07-06T22:00:00.000Z
 | | reportFormat: XLS
 | | locale: en_US
 | | enabled: true
 | | reportId: 03bc1eef-082e-46c1-b9a8-fe45fbc3b205
 | | parameters: 
 | | | queueIds: [
 | | | | 780807e6-83b9-44be-aff0-a41c37fab004
 | | | | 383ee5e5-5d8f-406c-ad53-835b69fe82c5
 | | | | d4fa7080-63d3-47ea-bcde-a4f89ae2870c
 | | | | 427b6226-cafd-4425-ba20-4e7dd8561088
 | | | | e0efef7f-b61a-4caf-b71b-002595c6899e
 | | | ]
 | | | mediaTypes: []
 | | 
 | | selfUri: /api/v2/analytics/reporting/schedules/c8c88e75-8f46-4b23-bae5-7ff8308ac00d
 | 
 | 
 | | id: 0cdd3c66-27ad-4d53-b64d-169611d6cc86
 | | quartzCronExpression: 0 14 * * * ?
 | | nextF

In [20]:
r2 = r['entities'][1].get('lastRun', None).get("reportUrl", None)
print(r2)

https://apps.mypurecloud.de/platform/api/v2/downloads/514ed69166673082


# Remove entries of a report

In [60]:
all_ids = [val['id'] for i, val in enumerate(r['entities'])]
print(all_ids)

['76beee10-531a-40ed-b483-c5b376e16431']


In [61]:
for ID in all_ids:
    url = f"https://api.{ENVIRONMENT}/api/v2/analytics/reporting/schedules/{ID}"
    response = requests.delete(url, headers=requestHeaders)

In [59]:
print(len(r['entities']))
type(r)

2


dict

# Read xlsx file from report and export it as DF

In [10]:
def data_frame_from_xlsx(report: dict, xls_file: str = "report_file.xls", remove: bool = True) -> pd.DataFrame:
    for var in report['entities']:
        if "lastRun" in var.keys():
            url = var["lastRun"]["reportUrl"]
            response = requests.get(url, headers=requestHeaders)
            print(response.json())
            
            # open(xls_file, "wb").write(response.content)
            # df = pd.read_excel(xls_file, header=6)
            # if remove: os.remove(xls_file)

            # return df

In [11]:
data = data_frame_from_xlsx(r, remove = True)
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# schedule a report

# Queue Metrics Interval Export

{
   "name": "",
   "quartzCronExpression": "",   (required)
   "nextFireTime": "",
   "dateCreated": "",
   "dateModified": "",
   "description": "",
   "timeZone": "",
   "timePeriod": "",
   "interval": "",               (required)
   "reportFormat": "",
   "locale": "",
   "enabled": true,
   "reportId": "",               (required)
   "parameters": {},
   "lastRun": {
      "name": "",
      "reportId": "",
      "runTime": "",
      "runStatus": "",
      "errorMessage": "",
      "runDurationMsec": 0,
      "reportUrl": "",
      "reportFormat": "",
      "scheduleUri": ""
   }
}

In [29]:
# quartzCronExpression (create firing schedules)
# sec min hour day month day_of_week year
# http://www.quartz-scheduler.org/documentation/quartz-2.3.0/tutorials/crontrigger.html
# report ID:
# Queue Metrics Summary Report: 03bc1eef-082e-46c1-b9a8-fe45fbc3b205


data_to_schedule = {
   "name": "test report 3",
   "quartzCronExpression": "0 15 14 * * ?",
   "description": "Export test 3",
   "timeZone": "Europe/Warsaw",
   "timePeriod": "USE_INTERVAL",
   "interval": "2022-07-012T22:00:00.000Z/2022-07-012T22:00:00.000Z",
   "reportFormat": "XLS",
   "enabled": True,
   "reportId": "03bc1eef-082e-46c1-b9a8-fe45fbc3b205"
}
print(json.dumps(data_to_schedule))

{"name": "test report 3", "quartzCronExpression": "0 15 14 * * ?", "description": "Export test 3", "timeZone": "Europe/Warsaw", "timePeriod": "USE_INTERVAL", "interval": "2022-07-012T22:00:00.000Z/2022-07-012T22:00:00.000Z", "reportFormat": "XLS", "enabled": true, "reportId": "03bc1eef-082e-46c1-b9a8-fe45fbc3b205"}


In [ ]:
requestHeaders = {
    "Authorization": f"{ response_json['token_type'] } { response_json['access_token']}",
    "Content-Type": "application/json"
}



In [ ]:
# RAPORT ID 03bc1eef-082e-46c1-b9a8-fe45fbc3b205